Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [380]:
from itertools import product
from random import random, randint, shuffle, seed, choice
import numpy as np
from scipy import sparse

from copy import copy
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue

In [381]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [382]:
num_points = 100    #100, 1_000, 5_000
num_sets = num_points
density = .3

In [383]:
x = make_set_covering_problem(num_points, num_sets, density)
SETS = x.toarray()
print("Element at row=42 and column=42:", x[42, 42])

Element at row=42 and column=42: True


In [384]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, num_points - 1)
    new_state[index] = not new_state[index]
    return new_state

In [385]:
def fitness(state):
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [SETS[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)]),
        )
    )
    return valid, -cost

In [386]:
current_state = [choice([False for _ in range(num_points)]) for _ in range(num_sets)]
cur_fit = fitness(current_state)
print(fitness(current_state))
steps = 1   # as number of evaluations

# hill climbing
for step in range(10_000):
    new_state = tweak(current_state)
    new_fit = fitness(new_state)
    steps += 1
    if new_fit > cur_fit:
        current_state = new_state
        cur_fit = new_fit
        print(fitness(current_state))

print("# steps: ",steps)

(0, 0)
(30, -1)
(59, -2)
(72, -3)
(84, -4)
(89, -5)
(91, -6)
(95, -7)
(97, -8)
(99, -9)
(100, -10)
(100, -9)
(100, -8)
# steps:  10001


In [387]:
current_state = [choice([False for _ in range(num_points)]) for _ in range(num_sets)]
cur_fit = fitness(current_state)
print(fitness(current_state))
steps = 1   # as number of evaluations

# steepest hill climbing
for step in range(100):
    new_state = tweak(current_state)
    new_fit = fitness(new_state)
    steps += 1
    for _ in range(20):
        tmp_state = tweak(current_state)
        tmp_fit = fitness(tmp_state)
        steps += 1
        if tmp_fit > new_fit:
            new_state = tmp_state
            new_fit = tmp_fit
            steps += 1
    if new_fit > cur_fit:
        current_state = new_state
        cur_fit = new_fit
        steps += 1
        print(fitness(current_state))

print("# steps: ",steps)

(0, 0)
(42, -1)
(68, -2)
(82, -3)
(90, -4)
(96, -5)
(99, -6)
(100, -7)
# steps:  2128


In [388]:
def covered(state):
    return reduce(
        np.logical_or,
        [SETS[i] for i, t in enumerate(state) if t],
        np.array([False for _ in range(num_points)]),
    )

def goal_check(state):
    return np.all(covered(state))

In [389]:
def path_search(state, n):  #greedy best-first (bounded)
    frontier = PriorityQueue()
    frontier.put((num_sets - np.sum(state), state))

    counter = 0
    _, current_state = frontier.get()
    
    while not goal_check(current_state) and counter<n:
        counter += 1
        for i, action in enumerate(current_state):
            if action==False:
                new_state = current_state
                new_state[i] = True
                frontier.put((num_sets - np.sum(new_state), new_state))
        _, current_state = frontier.get()
    return current_state

def tweak(state):
    max_steps = 5
    new_state = path_search(state, max_steps)
    #print("tweak\t",np.sum(state),np.sum(new_state))
    return new_state

In [390]:
max_steps = 10

current_state = [choice([False for _ in range(num_points)]) for _ in range(num_sets)]

cur_fit = fitness(current_state)
print(fitness(current_state))
steps = 1   # as number of evaluations

# steepest hill climbing
for step in range(100):
    new_state = tweak(current_state)
    new_fit = fitness(new_state)
    steps += 1
    for _ in range(20):
        tmp_state = tweak(current_state)
        tmp_fit = fitness(tmp_state)
        steps += 1
        if tmp_fit > new_fit:
            new_state = tmp_state
            new_fit = tmp_fit
            steps += 1
    if new_fit > cur_fit:
        current_state = new_state
        cur_fit = new_fit
        steps += 1
        print(fitness(current_state))

print("# steps: ",steps)

(0, 0)
(100, -100)
# steps:  2102
